### Import Libraries

In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
from tensorflow.keras import models
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras.applications import DenseNet121
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator


### Prepare the data

In [18]:
train_datagen = ImageDataGenerator(featurewise_center=False,
                                   featurewise_std_normalization=False,
                                   rescale=1/255,
                                   # preprocessing_function=fft,
                                   height_shift_range=0.2,
                                   width_shift_range=0.2,
                                   rotation_range=20,
                                   # zoom_range=0.05,
                                   # brightness_range=[0.9,1.1],
                                   # fill_mode='constant',
                                   horizontal_flip=True
                                                                            )
test_datagen = ImageDataGenerator(featurewise_center=False,
                                  featurewise_std_normalization=False,
                                  rescale=1/255
                                  # preprocessing_function=fft
                                                                            )

In [19]:
train_generator = train_datagen.flow_from_directory('/global/cfs/cdirs/m2571/Shinjan/Atomagined/Data_raw/Training_raw', 
                                                    target_size=(224, 224), 
                                                    class_mode= 'categorical', 
                                                    color_mode='rgb', 
                                                    batch_size=32,
                                                    interpolation ='lanczos'
                                                    )
validation_generator = test_datagen.flow_from_directory('/global/cfs/cdirs/m2571/Shinjan/Atomagined/Data_raw/Validation_raw', 
                                                        target_size=(224, 224), 
                                                        class_mode= 'categorical',
                                                        color_mode='rgb', 
                                                        batch_size=32,
                                                        interpolation ='lanczos'
                                                        )

Found 40718 images belonging to 7 classes.
Found 13574 images belonging to 7 classes.


In [7]:
x,_=next(train_generator)
print(x.shape)

(32, 224, 224, 1)


In [9]:
print(train_generator[0])

(array([[[[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        ...,

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]],

        [[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.        ],
         [0.        ],
         [0.        ]]],


       [[[0.        ],
         [0.        ],
         [0. 

In [14]:
train_set=tf.keras.utils.image_dataset_from_directory(
    '/global/cfs/cdirs/m2571/Shinjan/Atomagined/Data_raw/Training_raw',
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(250, 250),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
    # **kwargs
)

Found 40718 files belonging to 7 classes.


2023-01-09 02:32:02.040890: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-09 02:32:05.792572: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78993 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:03:00.0, compute capability: 8.0
2023-01-09 02:32:05.804247: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 78993 MB memory:  -> device: 1, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:41:00.0, compute capability: 8.0
2023-01-09 02:32:05.807387: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/devi

In [27]:
len(train_set)

1273

### Data Augmentation

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.02),
        layers.RandomWidth(0.2),
        layers.RandomHeight(0.2),
    ]
)

# Setting the state of the normalization layer.
data_augmentation.layers[0].adapt(train_generator)

### Build the Encoder

In [35]:
def create_encoder():
    resnet = keras.applications.ResNet50V2(
        include_top=False, weights=None, input_shape=(224,224,3), pooling="avg"
    )

    inputs = keras.Input(shape=input_shape)
    # augmented = data_augmentation(inputs)
    outputs = resnet(inputs)
    model = keras.Model(inputs=inputs, outputs=outputs, name="encoder")
    return model


encoder = create_encoder()
encoder.summary()

learning_rate = 0.001
batch_size = 265
hidden_units = 512
projection_units = 128
num_epochs = 50
dropout_rate = 0.5
temperature = 0.05
# input_shape=(224,224,3)

Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_23 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
Total params: 23,564,800
Trainable params: 23,519,360
Non-trainable params: 45,440
_________________________________________________________________


### Build the Classification model

In [28]:
def create_classifier(encoder, trainable=True):

    for layer in encoder.layers:
        layer.trainable = trainable

    inputs = keras.Input(shape=inp)
    features = encoder(inputs)
    features = layers.Dropout(dropout_rate)(features)
    features = layers.Dense(hidden_units, activation="relu")(features)
    features = layers.Dropout(dropout_rate)(features)
    outputs = layers.Dense(num_classes, activation="softmax")(features)

    model = keras.Model(inputs=inputs, outputs=outputs, name="classifier")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )
    return model

### Supervised contrastive learning loss function

In [30]:
class SupervisedContrastiveLoss(keras.losses.Loss):
    def __init__(self, temperature=1, name=None):
        super().__init__(name=name)
        self.temperature = temperature

    def __call__(self, labels, feature_vectors, sample_weight=None):
        # Normalize feature vectors
        feature_vectors_normalized = tf.math.l2_normalize(feature_vectors, axis=1)
        # Compute logits
        logits = tf.divide(
            tf.matmul(
                feature_vectors_normalized, tf.transpose(feature_vectors_normalized)
            ),
            self.temperature,
        )
        return tfa.losses.npairs_loss(tf.squeeze(labels), logits)


def add_projection_head(encoder):
    inputs = keras.Input(shape=(224,224,3))
    features = encoder(inputs)
    outputs = layers.Dense(projection_units, activation="relu")(features)
    model = keras.Model(
        inputs=inputs, outputs=outputs, name="encoder_with_projection-head"
    )
    return model

### Pretrain the encoder

In [31]:
encoder = create_encoder()

encoder_with_projection_head = add_projection_head(encoder)
encoder_with_projection_head.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=SupervisedContrastiveLoss(temperature),
    metrics=['acc',                       
                     tf.keras.metrics.AUC(name='auc')]
)

encoder_with_projection_head.summary()

# history = encoder_with_projection_head.fit(
#     x=x_train, y=y_train, batch_size=batch_size, epochs=num_epochs
# )

Model: "encoder_with_projection-head"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_21 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 encoder (Functional)        (None, 2048)              23564800  
                                                                 
 dense_2 (Dense)             (None, 128)               262272    
                                                                 
Total params: 23,827,072
Trainable params: 23,781,632
Non-trainable params: 45,440
_________________________________________________________________


In [32]:
es = EarlyStopping(monitor='val_auc', 
                   verbose=1, 
                   patience=5, 
                   min_delta=0.001, 
                   mode='max')
mc = ModelCheckpoint('/global/cfs/cdirs/m2571/Shinjan/Atomagined/ModelCheckpoint/Contrastive_trial1/', 
                     monitor='val_auc', 
                     verbose=1, 
                     save_best_only=True, 
                     mode='max',
                    save_weights_only=True)
rlr = ReduceLROnPlateau(monitor='val_auc',
                        mode='max',
                        factor=0.1,
                        patience=3)

In [33]:
class_weights = {0: (40718/819)/7,        #cubic
                1: (40718/2269)/7,        #hexagonal
                2: (40718/15953)/7,       #monoclinic
                3: (40718/9712)/7,        #orthorhombic
                4: (40718/2529)/7,        #tetragonal
                5: (40718/6085)/7,        #triclinic
                6: (40718/3351)/7}        #trigonal

In [34]:
history = encoder_with_projection_head.fit(train_generator, 
                                           batch_size=32, 
                                           epochs=50, 
                                           validation_data=validation_generator,
                                           class_weight = class_weights, 
                                           callbacks = [es, mc, rlr])

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.savefig('/global/cfs/cdirs/m2571/Shinjan/Atomagined/ModelCheckpoint/Contrastive_trial1/acc.png')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.savefig('/global/cfs/cdirs/m2571/Shinjan/Atomagined/ModelCheckpoint/Contrastive_trial1/loss.png')
plt.show()

plt.plot(history.history['auc'])
plt.plot(history.history['val_auc'])
plt.title('Model auc')
plt.ylabel('AUC')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.savefig('/global/cfs/cdirs/m2571/Shinjan/Atomagined/ModelCheckpoint/Contrastive_trial1/auc.png')
plt.show()

plt.plot(history.history['precision'])
plt.plot(history.history['val_precision'])
plt.title('Model precision')
plt.ylabel('Precision')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.savefig('/global/cfs/cdirs/m2571/Shinjan/Atomagined/ModelCheckpoint/Contrastive_trial1/precision.png')
plt.show()

plt.plot(history.history['recall'])
plt.plot(history.history['val_recall'])
plt.title('Model recall')
plt.ylabel('Recall')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.savefig('/global/cfs/cdirs/m2571/Shinjan/Atomagined/ModelCheckpoint/Contrastive_trial1/recall.png')
plt.show()

Epoch 1/50


2023-01-09 06:23:37.879029: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at segment_reduction_ops_impl.h:482 : INVALID_ARGUMENT: data.shape = [224] does not start with segment_ids.shape = [4096]
2023-01-09 06:23:37.879100: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at segment_reduction_ops_impl.h:482 : INVALID_ARGUMENT: data.shape = [224] does not start with segment_ids.shape = [4096]


InvalidArgumentError: Graph execution error:

Detected at node 'UnsortedSegmentSum' defined at (most recent call last):
    File "/global/common/software/nersc/pm-2022q4/sw/tensorflow/2.9.0/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/global/common/software/nersc/pm-2022q4/sw/tensorflow/2.9.0/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/global/common/software/nersc/pm-2022q4/sw/tensorflow/2.9.0/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/global/common/software/nersc/pm-2022q4/sw/tensorflow/2.9.0/lib/python3.9/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/global/common/software/nersc/pm-2022q4/sw/tensorflow/2.9.0/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 711, in start
      self.io_loop.start()
    File "/global/common/software